In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import spacy 
nlp = spacy.load("en_core_web_sm")
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.utils import to_categorical
from keras import layers
import numpy as np

User-defined Functions

In [2]:
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not (token.is_punct or token.is_stop) ]
    return ' '.join(tokens)

def find_max_len_string(texts):
    max_len = len(texts.loc[0])
    max_string = ''
    i = 0
    while i < len(df1):
        s = texts.loc[i]
        # print(s)
        if len(s) < max_len:
            max_len = len(s)
            max_string = s
        i += 1
    return  max_string

def findMinMax(arrays):
    max = None
    max_len = 0
    min_len = len(arrays[0])
    min = arrays[0]
    for array in arrays:
        size = len(array)
        if size > max_len:
            max = array
        if size < min_len:
            min = array
    return max, min

Read Data

In [4]:
df = pd.read_csv('../Assignments/archive/training.1600000.processed.noemoticon.csv', encoding = "cp1252")
first_row = df.columns
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
first_row_df = pd.DataFrame(data=None, columns = df.columns)
first_row_df.loc[0] = first_row
df = pd.concat([first_row_df,df])
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Choosing a user

In [5]:
user_frequency = df.user.value_counts()
preferred_users = user_frequency[user_frequency > 200]
preferred_users = preferred_users[user_frequency < 300]
print(preferred_users.index.values)

['SallytheShizzle' 'VioletsCRUK' 'mcraddictal' 'tsarnick' 'what_bugs_u'
 'Karen230683' 'DarkPiano' 'SongoftheOss' 'Jayme1988' 'keza34'
 'ramdomthoughts' 'shanajaca' 'wowlew' 'nuttychris' 'TraceyHewins'
 'thisgoeshere' 'Spidersamm' 'StDAY']


Let's choose user DarkPiano for this experiment

In [6]:
user = 'DarkPiano'
df1 = df[df.loc[:,"user"] == user]
df1 = df1.reset_index()
df1.shape

(236, 7)

Preprocessing the tweets

In [7]:
df1.text = df1.text.apply(preprocess)

Create training data

In [8]:
seq_length = 30

# Create a sequence of tokens
sequences = []
for i in range(len(df1)):
    tweet = df1.text.loc[i]

    for i in range(seq_length, len(tweet)):
        if len(tweet) > seq_length:
            seq = tweet[i-seq_length:i+1]
            sequences.append(seq)

# Convert sequences into numerical data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)

sequences = pad_sequences(sequences, maxlen=5, padding='pre')
sequences = np.array(sequences)

# Take the last integer of the arrays as y and rest as X
X = sequences[:,:-1]
y = sequences[:,-1]
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
y = to_categorical(y, num_classes=vocab_size)

1971


In [9]:
print(X.shape, y.shape)

(2363, 4) (2363, 1971)


Define model

In [10]:
model = Sequential([
    layers.Embedding(vocab_size, 50, input_length=4),
    layers.LSTM(150, return_sequences=True),
    layers.Dropout(0.1),
    layers.LSTM(100),
    layers.Dense(100, activation='relu'),
    layers.Dense(vocab_size, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

2023-05-13 16:38:52.470238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 50)             98550     
                                                                 
 lstm (LSTM)                 (None, 4, 150)            120600    
                                                                 
 dropout (Dropout)           (None, 4, 150)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 1971)              199071    
                                                                 
Total params: 528,721
Trainable params: 528,721
Non-trai